<a href="https://colab.research.google.com/github/Zafar-Beg/SnowHydrology/blob/main/GlacierMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-zafar')

In [9]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
glacier = ee.FeatureCollection("GLIMS/current")
geometry = ee.Geometry.Polygon(
        [[[5.501897369938105, 48.198582724702206],
          [5.501897369938105, 43.36861058284113],
          [15.114934479313105, 43.36861058284113],
          [15.114934479313105, 48.198582724702206]]])

Map.addLayer(geometry,{},'ROI')
Map.centerObject(geometry, 8)
Map

Map(bottom=812.0, center=[45.84928242931754, 10.308415924625576], controls=(WidgetControl(options=['position',…

In [12]:
glacierBounds=glacier.filterBounds(geometry).randomColumn('noise').sort('noise').limit(10000)
Map.addLayer(glacierBounds,{},'glacierROI')
Map.centerObject(glacierBounds, 8)
Map

Map(bottom=3240.0, center=[46.356235223016206, 8.83429154481984], controls=(WidgetControl(options=['position',…

In [17]:
containedgeometry = ee.Feature(ee.FeatureCollection(glacierBounds.map(lambda ft: ft.bounds())).union(1000).first()).convexHull().geometry().buffer(1000)


Map.addLayer(containedgeometry, {}, 'glaciercontainedROI')
Map.centerObject(containedgeometry, 8)
Map



Map(bottom=3280.0, center=[46.05973151308745, 9.627724647751743], controls=(WidgetControl(options=['position',…

In [20]:
glacier = glacier.map(lambda ft: ft.set('year', ee.Number.parse(ee.String(ft.get('src_date')).slice(0, 4))))

In [53]:
glac_if = ee.Filter.And(
    ee.Filter.maxDifference(
        difference=10000, leftField='year', rightField='year'
    ),
    ee.Filter.lessThan(leftField='year', rightField='year'),
    ee.Filter.equals(leftField='glac_id', rightField='glac_id')
)

In [57]:
saveBestJoin = ee.Join.saveBest(matchKey= 'nextGlacierRecord', measureKey= 'timeDiff')

In [62]:
intersectJoined = saveBestJoin.apply(glacier, glacier, glac_if);
intersectJoined=intersectJoined.map(lambda ft: ft.set('follwingObservationYear',ee.Feature(ft.get('nextGlacierRecord')).get('year')))

In [64]:
firstNoIceObservation=intersectJoined.sort('year',False).reduceToImage(['follwingObservationYear'], ee.Reducer.firstNonNull());


In [65]:
lastIceObservation=glacier.reduceToImage(['year'], ee.Reducer.max())

In [66]:
image=firstNoIceObservation.rename('fNIO').addBands(lastIceObservation.rename('lIO'))

In [67]:
Map.addLayer(image,{min: 1950, max:2020})
Map

Map(bottom=23600.0, center=[46.057985244793024, 9.626770019531252], controls=(WidgetControl(options=['position…